In [1]:
import os

In [2]:
%pwd

'd:\\VTION\\Projects\\Deep-Learning\\Perceptron\\research'

In [3]:
os.chdir("../")

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [5]:
from perceptronBasics.constants import *
from perceptronBasics.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        # self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )

        return data_transformation_config

In [7]:
import os
from perceptronBasics.logging import logger
import pandas as pd

In [8]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def handle_missing_values(self, row):
        row.fillna(row.mean(), inplace=True)
        return row

    def handle_duplicate_values(self, row):
        row.drop_duplicates(inplace=True)
        return row

    def convert(self):
        dataset = pd.read_csv(self.config.data_path)
        transformed_dataset = dataset.apply(self.handle_missing_values, axis=1)
        transformed_dataset = dataset.apply(self.handle_duplicate_values, axis=0)
        transformed_dataset.to_csv(os.path.join(self.config.root_dir,"transformed_dataset.csv"))

In [9]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config= data_transformation_config)
    data_transformation = data_transformation.convert()
except Exception as e:
    print(e)

[2024-04-05 04:26:03,015: INFO: common: Directory artifacts already exists]
[2024-04-05 04:26:03,016: INFO: common: Directory artifacts/data_transformation already exists]
